In [1]:
import pandas as pd



import datetime


from AvailableErrors import *
from Andrii.Dir_Task_JAN_FEB.tools_functions import createTimeDifference

In [2]:
global _WEEK_DICT, _HOLIDAY
_WEEK_DICT = {0:'ПН', 1:'ВТ', 2:'СР', 3:'ЧТ', 4:'ПТ', 5:'СБ', 6:'ВС'}
_HOLIDAY_MARKERS = [5,6]

In [20]:
class StrategyHorizon:
    @classmethod
    def StrategyIndex(cls):
        return 'StrategyHorizon'

    def __init__(self):
        self._start = None
        self._end = None
        self.input_data = None

        self.filters = list()

        self.after_filters = list()
        pass

    def input_params(self, start_horizon:datetime.timedelta, end_horizon: datetime.timedelta):
        self._start = start_horizon
        self._end = end_horizon

    def apply_filters(self, single_filter):
        if self.StrategyIndex not in single_filter.compatible:
            raise StrategyErrors().UnCompatibleFilter(Strategy=self, Filter=single_filter)
        self.filters.append(single_filter)

    def transfer_data(self, data: pd.DataFrame):
        if type(data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.input_data = data

    def _making_horizon(self):
        for dot_on_all_data in self.input_data.index:
            logical_filter = True
            #   SELECT BORDERS
            buffer_data = self.input_data.loc[dot_on_all_data + self._start: dot_on_all_data + self._end]
            for single_filter in self.filters:
                FILTER = single_filter
                FILTER.apply_buffer_data(buffer_data=buffer_data)
                if not FILTER.condition():
                    logical_filter = False
                    break


            if logical_filter:
                self.after_filters.append(buffer_data)

"""================================================================================================================================================="""
class CentralDotFilter:
    def __init__(self):
        self.compatible = [StrategyHorizon.StrategyIndex]
        self.buffer_data = None


    @classmethod
    def FilterIndex(cls):
        return 'CentralDotFilter'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = buffer_data.copy()

    def condition(self, CentralDotCoords):
        pass

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class ConsistingTime:
    def __init__(self,*consist_time):
        self.compatible = [StrategyHorizon.StrategyIndex]
        self.buffer_data = None
        for _ in consist_time:
            if type(_) != pd._libs.tslibs.timedeltas.Timedelta:
                raise FilterErrors().WrongShiftedType()
        self.consist_time = [_ for _ in consist_time]

    @classmethod
    def FilterIndex(cls):
        return 'ConsistingTime'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()

        ZERO_TIME = buffer_data.index[0].to_numpy()
        ZERO_TIME = ZERO_TIME.astype('datetime64[s]').item().date()
        self.buffer_data = buffer_data.index - pd.to_datetime(ZERO_TIME)




        #self.buffer_data = buffer_data.copy()

        #if type(self.buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
        #    raise FilterErrors().WrongTimeType()
        #ZERO_TIME = self.buffer_data.index[0].day

        #   1231232self.buffer_data.index = [pd.Timedelta(days=_.day - ZERO_TIME, hours=_.hour, minutes=_.minute) for _ in self.buffer_data.index]

        #self.buffer_data.index = [pd.Timedelta(days=_.day - ZERO_TIME, hours=_.hour, minutes=_.minute) for _ in self.buffer_data.index]

        return None

    def condition(self):
        for timedelta in self.consist_time:
            #if not timedelta in self.buffer_data.index:
            if not timedelta in self.buffer_data:
                return False

        return True

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class NoHolidays:
    def __init__(self):
        self.compatible = [StrategyHorizon.StrategyIndex]
        self.buffer_data = None


    @classmethod
    def FilterIndex(cls):
        return 'NoHolidays'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = buffer_data.copy()
        return None

    def condition(self):
        for holiday in self.buffer_data.index.dayofweek:
            if holiday in _HOLIDAY_MARKERS:
                return False

        return True
"""================================================================================================================================================="""

"""
horizon = StrategyHorizon()
constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)
nodolidays = NoHolidays()
horizon.apply_filters(nodolidays)
horizon.transfer_data(d.iloc[:1000])
horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))
horizon._making_horizon()
horizon.after_filters
"""
d = pd.read_csv('zip_data/AUDCAD.csv', index_col=0)
d.index = pd.to_datetime(d.index)
d = d.resample('1T').first()
d = d.iloc[:150_00]
horizon = StrategyHorizon()
constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)
nodolidays = NoHolidays()
horizon.apply_filters(nodolidays)
horizon.transfer_data(d)
horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))
horizon._making_horizon()
horizon.after_filters

[                         open      high       low     close
 time                                                       
 2018-11-01 15:00:00  0.939375  0.939760  0.939285  0.939585
 2018-11-01 16:00:00  0.939495  0.939565  0.939375  0.939445
 2018-11-01 17:00:00  0.939295  0.939895  0.939270  0.939865
 2018-11-01 18:00:00  0.942600  0.942680  0.942535  0.942640
 2018-11-01 19:00:00  0.942275  0.942335  0.942075  0.942140
 2018-11-01 20:00:00  0.942205  0.942270  0.942025  0.942195
 2018-11-01 21:00:00  0.942930  0.943070  0.942895  0.943030
 2018-11-01 22:00:00  0.943120  0.943165  0.943050  0.943065
 2018-11-01 23:00:00  0.943680  0.943690  0.943505  0.943515
 2018-11-02 00:00:00  0.942955  0.943045  0.942955  0.943040
 2018-11-02 01:00:00  0.942280  0.942380  0.942265  0.942335
 2018-11-02 02:00:00  0.942200  0.942445  0.942175  0.942385
 2018-11-02 03:00:00  0.943185  0.943295  0.943160  0.943205
 2018-11-02 04:00:00  0.943615  0.943615  0.943210  0.943225
 2018-11-02 05:00:00  0.

In [ ]:
d = pd.read_csv('zip_data/AUDCAD.csv', index_col=0)
d.index = pd.to_datetime(d.index)
d = d.resample('1H').first()
d = d.iloc[:150_0]
horizon = StrategyHorizon()
constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)
#nodolidays = NoHolidays()
#horizon.apply_filters(nodolidays)
horizon.transfer_data(d)
horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))
horizon._making_horizon()
horizon.after_filters

In [ ]:
class ConsistingTime:
    def __init__(self,*consist_time):
        self.compatible = [StrategyHorizon.StrategyIndex]
        self.buffer_data = None
        for _ in consist_time:
            if type(_) != pd._libs.tslibs.timedeltas.Timedelta:
                raise FilterErrors().WrongShiftedType()
        self.consist_time = [_ for _ in consist_time]

    @classmethod
    def FilterIndex(cls):
        return 'ConsistingTime'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()

        ZERO_TIME = buffer_data.index[0].to_numpy()
        ZERO_TIME = ZERO_TIME.astype('datetime64[s]').item().date()
        self.buffer_data = buffer_data.index - pd.to_datetime(ZERO_TIME)




        #self.buffer_data = buffer_data.copy()

        #if type(self.buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
        #    raise FilterErrors().WrongTimeType()
        #ZERO_TIME = self.buffer_data.index[0].day

        #   1231232self.buffer_data.index = [pd.Timedelta(days=_.day - ZERO_TIME, hours=_.hour, minutes=_.minute) for _ in self.buffer_data.index]

        #self.buffer_data.index = [pd.Timedelta(days=_.day - ZERO_TIME, hours=_.hour, minutes=_.minute) for _ in self.buffer_data.index]

        return None

    def condition(self):
        for timedelta in self.consist_time:
            #if not timedelta in self.buffer_data.index:
            if not timedelta in self.buffer_data:
                return False

        return True
